In [1]:
# Import the dependencies.
from pathlib import Path
from sqlalchemy import create_engine, text
import sqlite3
import csv
import pandas as pd

In [2]:
    # Connect to SQLite
    sqliteConnection = sqlite3.connect('Covid2020_DB.sqlite')
    cursor = sqliteConnection.cursor()

In [3]:
cursor.execute('''
DROP VIEW COVID2020_TRAINING
''')

In [4]:
    # Create temp view for people that died (211270)
    cursor.execute('''
    CREATE VIEW COVID2020_TRAINING AS
    select * from COVID2020
    WHERE FECHA_DEF <  '9999-99-99'
    ;''')

In [5]:
cursor.execute('''
SELECT * FROM COVID2020_TRAINING
''')

In [6]:
cursor.execute('''
DROP VIEW COVID2020_TRAINING2
''')

In [7]:
    # Create temp view for people that did not die (211270)
    cursor.execute('''
    CREATE VIEW COVID2020_TRAINING2 AS
    select * from COVID2020
    WHERE FECHA_DEF >=  '9999-99-99'
    LIMIT 211270
    ;''')

In [8]:
cursor.execute('''
SELECT * FROM COVID2020_TRAINING2
''')

In [9]:
sql_query = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM COVID2020_TRAINING
                               ''', sqliteConnection)

COVID2020_filtered_death_df = pd.DataFrame(sql_query, columns = ["FECHA_ACTUALIZACION","ID_REGISTRO","ORIGEN","SECTOR","ENTIDAD_UM","SEXO","ENTIDAD_NAC","ENTIDAD_RES","MUNICIPIO_RES","TIPO_PACIENTE","FECHA_INGRESO","FECHA_SINTOMAS","FECHA_DEF","INTUBADO","NEUMONIA","EDAD","NACIONALIDAD","EMBARAZO","HABLA_LENGUA_INDIG","INDIGENA","DIABETES","EPOC","ASMA","INMUSUPR","HIPERTENSION","OTRA_COM","CARDIOVASCULAR","OBESIDAD","RENAL_CRONICA","TABAQUISMO","OTRO_CASO","TOMA_MUESTRA_LAB","RESULTADO_LAB","TOMA_MUESTRA_ANTIGENO","RESULTADO_ANTIGENO","CLASIFICACION_FINAL","MIGRANTE","PAIS_NACIONALIDAD","PAIS_ORIGEN","UCI"])

In [10]:
COVID2020_filtered_death_df.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2021-10-31,z4e838,1,12,9,2,15,15,95,2,...,99,1,2,2,97,7,99,México,97,2
1,2021-10-31,z1da2d,1,12,2,1,2,2,4,2,...,99,2,97,2,97,6,99,México,97,2
2,2021-10-31,z49a69,1,12,23,1,23,23,4,2,...,1,2,97,2,97,2,99,México,97,1
3,2021-10-31,1236ca,1,3,15,2,15,15,60,2,...,2,1,1,2,97,3,99,México,97,2
4,2021-10-31,12914d,1,12,32,1,32,32,56,2,...,2,1,1,2,97,3,99,México,97,2


In [11]:
sql_query2 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM COVID2020_TRAINING2
                               ''', sqliteConnection)

COVID2020_filtered_alive_df = pd.DataFrame(sql_query2, columns = ["FECHA_ACTUALIZACION","ID_REGISTRO","ORIGEN","SECTOR","ENTIDAD_UM","SEXO","ENTIDAD_NAC","ENTIDAD_RES","MUNICIPIO_RES","TIPO_PACIENTE","FECHA_INGRESO","FECHA_SINTOMAS","FECHA_DEF","INTUBADO","NEUMONIA","EDAD","NACIONALIDAD","EMBARAZO","HABLA_LENGUA_INDIG","INDIGENA","DIABETES","EPOC","ASMA","INMUSUPR","HIPERTENSION","OTRA_COM","CARDIOVASCULAR","OBESIDAD","RENAL_CRONICA","TABAQUISMO","OTRO_CASO","TOMA_MUESTRA_LAB","RESULTADO_LAB","TOMA_MUESTRA_ANTIGENO","RESULTADO_ANTIGENO","CLASIFICACION_FINAL","MIGRANTE","PAIS_NACIONALIDAD","PAIS_ORIGEN","UCI"])

In [12]:
COVID2020_filtered_alive_df.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2021-10-31,z4d6fe,1,6,24,1,24,24,28,1,...,2,1,1,2,97,3,99,México,97,97
1,2021-10-31,z2a3cf,1,6,24,2,24,24,28,1,...,1,2,97,2,97,6,99,México,97,97
2,2021-10-31,z1e370,1,12,14,1,14,14,85,1,...,2,1,2,2,97,7,99,México,97,97
3,2021-10-31,z3bf80,2,12,8,2,8,8,37,1,...,2,1,1,2,97,3,99,México,97,97
4,2021-10-31,zz7067,1,12,9,2,9,9,7,1,...,2,1,2,2,97,7,99,México,97,97


In [13]:
    # Commit work and close connection
    sqliteConnection.commit()

In [14]:
cursor.close()

In [34]:
#Combine/concat data frames
COVID2020_training_data_df=pd.concat([COVID2020_filtered_alive_df,COVID2020_filtered_death_df])

In [35]:
#Reset index
COVID2020_training_data_df=COVID2020_training_data_df.reset_index()
COVID2020_training_data_df=COVID2020_training_data_df.drop(columns='index')
COVID2020_training_data_df

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2021-10-31,z4d6fe,1,6,24,1,24,24,28,1,...,2,1,1,2,97,3,99,México,97,97
1,2021-10-31,z2a3cf,1,6,24,2,24,24,28,1,...,1,2,97,2,97,6,99,México,97,97
2,2021-10-31,z1e370,1,12,14,1,14,14,85,1,...,2,1,2,2,97,7,99,México,97,97
3,2021-10-31,z3bf80,2,12,8,2,8,8,37,1,...,2,1,1,2,97,3,99,México,97,97
4,2021-10-31,zz7067,1,12,9,2,9,9,7,1,...,2,1,2,2,97,7,99,México,97,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422535,2021-10-31,94d648,2,9,9,1,15,15,58,2,...,2,2,97,2,97,2,99,México,97,2
422536,2021-10-31,bb6edb,2,9,9,2,15,15,25,2,...,1,2,97,2,97,2,99,México,97,2
422537,2021-10-31,7d32d9,2,9,9,2,9,9,7,2,...,1,2,97,2,97,2,99,México,97,1
422538,2021-10-31,660ba7,1,4,9,1,9,9,7,2,...,2,2,97,2,97,2,99,México,97,2


In [36]:
COVID2020_training_data_df=COVID2020_training_data_df.drop(columns=['FECHA_ACTUALIZACION','ID_REGISTRO','NACIONALIDAD','TOMA_MUESTRA_LAB','TOMA_MUESTRA_ANTIGENO','PAIS_NACIONALIDAD','PAIS_ORIGEN'])
COVID2020_training_data_df

,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,...,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO_LAB,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,UCI
0,1,6,24,1,24,24,28,1,2020-12-06,2020-11-30,...,2,2,2,2,2,1,97,3,99,97
1,1,6,24,2,24,24,28,1,2020-02-20,2020-02-18,...,2,2,2,2,1,97,97,6,99,97
2,1,12,14,1,14,14,85,1,2020-04-22,2020-04-18,...,2,2,2,2,2,2,97,7,99,97
3,2,12,8,2,8,8,37,1,2020-07-28,2020-07-20,...,2,2,2,2,2,1,97,3,99,97
4,1,12,9,2,9,9,7,1,2020-08-18,2020-08-17,...,2,2,2,2,2,2,97,7,99,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422535,2,9,9,1,15,15,58,2,2020-12-17,2020-12-14,...,2,2,2,2,2,97,97,2,99,2
422536,2,9,9,2,15,15,25,2,2020-04-27,2020-04-15,...,2,2,2,2,1,97,97,2,99,2
422537,2,9,9,2,9,9,7,2,2020-12-05,2020-12-01,...,2,2,2,2,1,97,97,2,99,1
422538,1,4,9,1,9,9,7,2,2020-09-11,2020-09-09,...,2,2,2,2,2,97,97,2,99,2
